In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os

#加载和预处理数据
transform = transforms.Compose([transforms.ToTensor()])
#transforms.ToTensor(): 将PIL图像或NumPy数组转换为PyTorch张量，并且会将图像的像素值从范围 [0, 255]缩放到 [0.0, 1.0]。transforms.Normalize((0.5,), (0.5,)): 将张量的每个通道的像素值进行标准化。像素值从 [0, 1] 归一化到 [-1, 1]。具体公式是：(x - mean) / std，其中mean和std是给定的均值和标准差。
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
#torchvision.datasets.MNIST 是一个用于加载MNIST数据集的类。

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)
#torch.utils.data.DataLoader 是PyTorch用于加载数据集的类。它能够处理批量数据并进行多线程加载。批次的数据量为100,训练随机打乱，测试不打乱。

In [30]:
# 定义神经网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()#28*28二维图像展平成一维向量，size=784
        self.fc1 = nn.Linear(28*28, 30)#784维映射到128维
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)#舍去50%的神经元
        self.fc2 = nn.Linear(30, 10)#128维映射导10维

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
# 实例化模型、定义损失函数和优化器
model = Net()
criterion = nn.CrossEntropyLoss()#交叉熵损失函数
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [31]:
# 训练模型
num_epochs = 50

for epoch in range(num_epochs):
    model.train()#将模型设置为训练模式，以便在前向传播时启用 Dropout 和 BatchNorm
    running_loss = 0.0
    for inputs, labels in train_loader:
        #inputs_noise1 = add_noise(inputs)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # 在测试集上验证模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():#在验证过程中禁用梯度计算，以提高计算速度和节省内存。
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

print('Finished Training')

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import math
# 加载和预处理数据
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

class Custom(nn.Module):  # 激活函数含噪声参数
    def __init__(self):
        super().__init__()
        self.sigma = nn.Parameter(torch.randn(1))  # 1个可学习参数 sigma
    def forward(self, input):
        temp = input * (1 + torch.erf(input / self.sigma)) / 2  # 前向传播
        return temp

class NetGELU(nn.Module):
    def __init__(self):
        super(NetGELU, self).__init__()
        self.flatten = nn.Flatten()  # 28*28二维图像展平成一维向量，size=784
        self.fc1 = nn.Linear(28*28, 30)  # 784维映射到50维
        self.custom1 = Custom()  # 自定义激活函数层
        self.dropout = nn.Dropout(0.5)  # 舍去50%的神经元
        self.fc2 = nn.Linear(30, 10)  # 50维映射到10维

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.custom1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 实例化模型、定义损失函数和优化器
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NetGELU().to(device)
criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 50
z = model.custom1.sigma
nn.init.constant_(z, 4)
#std_dev = 0.01  # 高斯噪声的标准差

for epoch in range(num_epochs):
    model.train()  # 将模型设置为训练模式，以便在前向传播时启用 Dropout 和 BatchNorm
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        # with torch.no_grad():
        #     for param in model.parameters():
        #         param.grad += std_dev * torch.randn(param.grad.shape).to(param.grad.device)
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # 在测试集上验证模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # 在验证过程中禁用梯度计算，以提高计算速度和节省内存。
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

print('Finished Training')

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np

# 加载和预处理数据
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

# 定义自定义激活函数
class CustomActivation(nn.Module):
    def __init__(self, sigma=1.0):
        super(CustomActivation, self).__init__()
        self.sigma = sigma

    def forward(self, x):
        return torch.where(x >= 0, x, x * torch.exp(x / self.sigma))

# 定义神经网络模型
class Net(nn.Module):
    def __init__(self, sigma=1.0):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 30)
        self.custom_activation = CustomActivation(sigma)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(30, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.custom_activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 实例化模型、定义损失函数和优化器
sigma = 9.0  # 你可以根据需要调整sigma的值
model = Net(sigma=sigma)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 训练模型
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # 在测试集上验证模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

print('Finished Training')


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np

# 加载和预处理数据
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

# 自定义激活函数
class CustomActivation(nn.Module):
    def __init__(self, sigma=1.0):
        super(CustomActivation, self).__init__()
        self.sigma = sigma

    def forward(self, x):
        return torch.where(x >= 0, x, x * torch.exp(-x**2 / (2 * self.sigma**2)))

# 定义神经网络模型
class Net(nn.Module):
    def __init__(self, sigma=1.0):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 30)
        self.custom_activation = CustomActivation(sigma)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(30, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.custom_activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 实例化模型、定义损失函数和优化器
sigma = 9  # 你可以根据需要调整sigma的值
model = Net(sigma=sigma)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 训练模型
num_epochs = 50
for epoch in range(num_epochs):
    model.train()  # 将模型设置为训练模式，以便在前向传播时启用 Dropout 和 BatchNorm
    running_loss = 0.0
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # 在测试集上验证模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # 在验证过程中禁用梯度计算，以提高计算速度和节省内存。
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

print('Finished Training')
